In [62]:
#imports
import pandas as pd

In [63]:
#load original dfs
original_df = pd.read_csv("../data/aggregated_data.csv")

In [70]:
#methods to create change_by_year data

def generateChangeDataFeatureColumns(df, column):
    df[f'{column}_val_change'] = None 
    df[f'{column}_perc_change'] = None 
    # Create an empty DataFrame with the same columns as the original one 
    new_df = pd.DataFrame(columns=df.columns) 
    # Get unique states and years 
    states = df["STATE"].unique() 
    years = df["YEAR"].unique() 
    # Iterate over states and years 
    for state in states: 
        for year in years: 
            if year - 1 in years: 
                row = df[(df['STATE'] == state) & (df['YEAR'] == year)].copy() 
                last_years_row = df[(df['STATE'] == state) & (df['YEAR'] == year - 1)] 
                if not row.empty and not last_years_row.empty: 
                    current_value = row[column].values[0] 
                    last_value = last_years_row[column].values[0] 
                    # Calculate the value change and percentage change 
                    row[f'{column}_val_change'] = current_value - last_value 
                    row[f'{column}_perc_change'] = (current_value - last_value) / last_value 
                    new_df = pd.concat([new_df, row]) 
                else: print(f"No data found for State: {state}, Year: {year} or Year: {year - 1}") 
            else: 
                print(f'No previous year found for Year: {year} in State: {state}')

    return new_df



In [71]:
test_df = original_df.copy()
test_df = generateChangeDataFeatureColumns(test_df, "mean_ADHD")
print("showing data")
print(len(test_df["mean_ADHD_val_change"].tolist()))



C:\Users\johnb\AppData\Local\Temp\ipykernel_35712\635169708.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, row])


No previous year found for Year: 2013 in State: AL
No data found for State: AK, Year: 2019 or Year: 2018
No data found for State: AK, Year: 2018 or Year: 2017
No data found for State: AK, Year: 2017 or Year: 2016
No data found for State: AK, Year: 2016 or Year: 2015
No data found for State: AK, Year: 2015 or Year: 2014
No data found for State: AK, Year: 2014 or Year: 2013
No previous year found for Year: 2013 in State: AK
No data found for State: AZ, Year: 2020 or Year: 2019
No data found for State: AZ, Year: 2019 or Year: 2018
No previous year found for Year: 2013 in State: AZ
No previous year found for Year: 2013 in State: AR
No previous year found for Year: 2013 in State: CA
No previous year found for Year: 2013 in State: CO
No previous year found for Year: 2013 in State: CT
No previous year found for Year: 2013 in State: DE
No data found for State: DC, Year: 2014 or Year: 2013
No previous year found for Year: 2013 in State: DC
No previous year found for Year: 2013 in State: FL
No d

In [72]:
row = test_df[(test_df['STATE'] == 'CT') & (test_df['YEAR'] == 2018)]
print(test_df['mean_ADHD_perc_change'])
val_change = row['mean_ADHD_val_change'].values[0] 
perc_change = row['mean_ADHD_perc_change'].values[0] 
val_2018 = row["mean_ADHD"].values[0]
print(f"Value Change for AL in 2018: {val_change}") 
print(f"Percentage Change for AL in 2018: {perc_change}")

row_2 = test_df[(test_df['STATE'] == 'AL') & (test_df['YEAR'] == 2017)]
val_2017 = row_2["mean_ADHD"].values[0]

print(f"2017 : {val_2017}, 2018 : {val_2018}")

0     -0.024138
47     0.050725
91    -0.028169
135    0.100775
179   -0.047970
         ...   
134   -0.004367
178    0.040909
221   -0.047619
263    0.004348
307    0.116505
Name: mean_ADHD_perc_change, Length: 300, dtype: float64
Value Change for AL in 2018: -0.3333333400000029
Percentage Change for AL in 2018: -0.01408450732196005
2017 : 23.66666667, 2018 : 23.33333333
